<a href="https://colab.research.google.com/github/yashveersinghsohi/Kaggle_Competitions/blob/HousePrice/House%20Prices/Submissions/Sub%205%20-%20Stacking/Sub_5_Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Required Packages

In [31]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import BaggingRegressor

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.ensemble import StackingRegressor

# Reading Data

In [2]:
url = "https://raw.githubusercontent.com/yashveersinghsohi/Kaggle_Competitions/HousePrice/House%20Prices/Submissions/Data/"

train_df = pd.read_csv(url + "train.csv")
test_df = pd.read_csv(url + "test.csv")

train_features_df = pd.read_csv(url + "train_features_df.csv")
test_features_df = pd.read_csv(url + "test_features_df.csv")

In [3]:
train_features_df.head()

,OverallQual,ExterQual,Foundation,BsmtQual,KitchenQual,FireplaceQu,GarageFinish,YearDiff,YearRemodAdd,1stFlrSF,GrLivArea,FullBath,GarageCars,Neighborhood,MasVnrType,HeatingQC,Fireplaces,MasVnrArea,GarageYrBlt,HouseStyle,Exterior1st,Exterior2nd,BsmtExposure,BsmtFinType1,GarageType,OverallCond,LotArea,BsmtFinSF1,LotFrontage,2ndFlrSF,SalePrice,no_garage,is_BsmtFinSF1_0,is_2ndFlrSF_0
0,7,2,6,3,2,1,2,2.472136,87.509776,6.752270,7.444249,2,2,16,2,4,0,196.0,2003,6,11,13,1,6,5,8,9.041922,706,4.174387,854,208500,0,0,0
1,6,1,3,3,1,3,2,9.135529,86.904443,7.140453,7.140453,2,2,19,1,4,1,0.0,1976,4,6,4,4,4,5,4,9.169518,978,4.382027,0,181500,0,0,1
2,7,2,6,3,2,3,2,3.291503,87.487429,6.824374,7.487734,2,2,16,2,4,1,162.0,2001,6,11,13,2,6,5,8,9.328123,486,4.219508,866,223500,0,0,0
3,7,1,2,2,2,4,1,17.078784,86.769364,6.867974,7.448334,1,3,18,1,3,1,0.0,1998,6,5,3,1,4,2,8,9.164296,216,4.094345,756,140000,0,0,0
4,8,2,6,3,2,3,2,3.656854,87.442719,7.043160,7.695303,2,3,23,2,4,1,350.0,2000,6,11,13,3,6,5,8,9.565214,655,4.430817,1053,250000,0,0,0


In [4]:
test_features_df.head()

,OverallQual,ExterQual,Foundation,BsmtQual,KitchenQual,FireplaceQu,GarageFinish,YearDiff,YearRemodAdd,1stFlrSF,GrLivArea,FullBath,GarageCars,Neighborhood,MasVnrType,HeatingQC,Fireplaces,MasVnrArea,GarageYrBlt,HouseStyle,Exterior1st,Exterior2nd,BsmtExposure,BsmtFinType1,GarageType,OverallCond,LotArea,BsmtFinSF1,LotFrontage,2ndFlrSF,no_garage,is_BsmtFinSF1_0,is_2ndFlrSF_0
0,5,1,3,2,1,1,1,12.000000,86.566359,6.797940,6.797940,1,1,9,1,2,0,0.0,7.581210,4,11,13,1,3,5,6,9.360655,468.0,4.382027,0,0,0,1
1,6,1,3,2,2,1,1,12.422205,86.498588,7.192182,7.192182,1,1,9,2,2,0,108.0,7.579679,4,5,2,1,4,5,6,9.565704,923.0,4.394449,0,0,0,1
2,5,1,6,3,1,3,3,5.211103,87.397987,6.833032,7.395722,2,2,13,1,3,1,0.0,7.599401,6,11,13,1,6,5,5,9.534595,791.0,4.304065,701,0,0,0
3,6,1,6,2,2,4,3,4.928203,87.397987,6.830874,7.380256,2,2,13,2,4,1,20.0,7.599902,6,11,13,1,6,5,6,9.208138,602.0,4.356709,678,0,0,0
4,8,2,6,3,2,1,2,6.485281,87.263654,7.154615,7.154615,2,2,22,1,4,0,0.0,7.596894,4,8,8,1,4,5,5,8.518193,263.0,3.761200,0,0,0,1


# Train-Test Split

In [5]:
def split_data(df, test_size = 0.2, random_state = 42):
    X, y = df.drop(["SalePrice"], axis = 1), df["SalePrice"]
    return train_test_split(X, y, test_size = test_size, random_state = random_state)

# Models with LB score < 0.14

## GBM Optimized 

In [21]:
X_train, X_test, y_train, y_test = split_data(train_features_df)
gbm = GradientBoostingRegressor(max_depth = 6, 
                                max_features = 12, 
                                min_samples_leaf = 10, 
                                min_samples_split = 25, 
                                subsample = 1.0, 
                                learning_rate = 0.001, 
                                n_estimators = 8000, 
                                random_state = 42)

gbm_fit = gbm.fit(X_train, np.log(y_train))
gbm_preds = np.exp(gbm_fit.predict(X_test))
gbm_rmsle = np.sqrt(mean_squared_log_error(y_true = y_test, y_pred = gbm_preds))

gbm_pipe_fit = make_pipeline(RobustScaler(), gbm).fit(X_train, np.log(y_train))
gbm_pipe_preds = np.exp(gbm_pipe_fit.predict(X_test))
gbm_pipe_rmsle = np.sqrt(mean_squared_log_error(y_true = y_test, y_pred = gbm_pipe_preds))

In [22]:
gbm_rmsle

0.13980079971568782

In [23]:
gbm_pipe_rmsle

0.13979032496498517

## XGBoost Optimized

In [39]:
X_train, X_test, y_train, y_test = split_data(train_features_df)
xgb = XGBRegressor(random_state = 42, 
                   max_depth = 5, 
                   subsample = 0.9, 
                   colsample_bytree = 0.5, 
                   gamma = 0, 
                   reg_alpha = 0.1, 
                   reg_lambda = 0.1, 
                   learning_rate = 0.05, 
                   n_estimators = 600)

xgb_fit = xgb.fit(X_train, np.log(y_train), 
                  early_stopping_rounds = 150, 
                  eval_metric = "rmse", 
                  eval_set = [(X_test, np.log(y_test))])
xgb_preds = np.exp(xgb_fit.predict(X_test))
xgb_rmsle = np.sqrt(mean_squared_log_error(y_true = y_test, y_pred = xgb_preds))

xgb_pipe_fit = make_pipeline(RobustScaler(), xgb).fit(X_train, np.log(y_train))
xgb_pipe_preds = np.exp(xgb_pipe_fit.predict(X_test))
xgb_pipe_rmsle = np.sqrt(mean_squared_log_error(y_true = y_test, y_pred = xgb_pipe_preds))

[11:28:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:10.93
Will train until validation_0-rmse hasn't improved in 150 rounds.
[1]	validation_0-rmse:10.3824
[2]	validation_0-rmse:9.86262
[3]	validation_0-rmse:9.36802
[4]	validation_0-rmse:8.89888
[5]	validation_0-rmse:8.45386
[6]	validation_0-rmse:8.03088
[7]	validation_0-rmse:7.62897
[8]	validation_0-rmse:7.24781
[9]	validation_0-rmse:6.88551
[10]	validation_0-rmse:6.54033
[11]	validation_0-rmse:6.2135
[12]	validation_0-rmse:5.903
[13]	validation_0-rmse:5.60721
[14]	validation_0-rmse:5.32672
[15]	validation_0-rmse:5.05982
[16]	validation_0-rmse:4.80658
[17]	validation_0-rmse:4.56683
[18]	validation_0-rmse:4.339
[19]	validation_0-rmse:4.12219
[20]	validation_0-rmse:3.91629
[21]	validation_0-rmse:3.72052
[22]	validation_0-rmse:3.53504
[23]	validation_0-rmse:3.35855
[24]	validation_0-rmse:3.19121
[25]	validation_0-rmse:3.0322
[26]	validation_0-r

In [40]:
xgb_rmsle

0.13768019296132125

In [41]:
xgb_pipe_rmsle

0.13773307427000914

## LightGBM Optimized

In [45]:
X_train, X_test, y_train, y_test = split_data(train_features_df)
lgbm = lgb.LGBMRegressor(random_state = 42, 
                         learning_rate = 0.005, 
                         n_estimators = 1200, 
                         max_depth = 9, 
                         max_leaves = 21, 
                         reg_alpha = 0.05, 
                         reg_lambda = 0.05, 
                         subsample = 0.6, 
                         colsample_bytree = 0.6)

lgbm_fit = lgbm.fit(X_train, np.log(y_train), 
                    early_stopping_rounds = 150, 
                    eval_metric = "rmse", 
                    eval_set = [(X_test, np.log(y_test))])
lgbm_preds = np.exp(lgbm_fit.predict(X_test))
lgbm_rmsle = np.sqrt(mean_squared_log_error(y_true = y_test, y_pred = lgbm_preds))

lgbm_pipe_fit = make_pipeline(StandardScaler(), lgbm).fit(X_train, np.log(y_train))
lgbm_pipe_preds = np.exp(lgbm_pipe_fit.predict(X_test))
lgbm_pipe_rmsle = np.sqrt(mean_squared_log_error(y_true = y_test, y_pred = lgbm_pipe_preds))

[1]	valid_0's rmse: 0.431576	valid_0's l2: 0.186258
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's rmse: 0.429966	valid_0's l2: 0.18487
[3]	valid_0's rmse: 0.428468	valid_0's l2: 0.183585
[4]	valid_0's rmse: 0.426825	valid_0's l2: 0.182179
[5]	valid_0's rmse: 0.425265	valid_0's l2: 0.18085
[6]	valid_0's rmse: 0.423601	valid_0's l2: 0.179438
[7]	valid_0's rmse: 0.422002	valid_0's l2: 0.178086
[8]	valid_0's rmse: 0.420496	valid_0's l2: 0.176817
[9]	valid_0's rmse: 0.418925	valid_0's l2: 0.175498
[10]	valid_0's rmse: 0.417415	valid_0's l2: 0.174235
[11]	valid_0's rmse: 0.415975	valid_0's l2: 0.173035
[12]	valid_0's rmse: 0.414506	valid_0's l2: 0.171815
[13]	valid_0's rmse: 0.412926	valid_0's l2: 0.170508
[14]	valid_0's rmse: 0.41144	valid_0's l2: 0.169282
[15]	valid_0's rmse: 0.409919	valid_0's l2: 0.168033
[16]	valid_0's rmse: 0.408411	valid_0's l2: 0.166799
[17]	valid_0's rmse: 0.407025	valid_0's l2: 0.165669
[18]	valid_0's rmse: 0.40549	valid_0's l2: 0.164

In [46]:
lgbm_rmsle

0.13911700302340038

In [47]:
lgbm_pipe_rmsle

0.13891269548335544

# Stacking Regressor

## Stack 1

**Base Models** used - 
- GBM with Robust Scaling
- XGBoost
- LightGBM with Standard Scaling

<br>

**Final Model** used - 
- RidgeCV(Default)

<br>

**Passthrough** = True

In [48]:
gbm_sr = make_pipeline(RobustScaler(), gbm)
xgb_sr = xgb
lgbm_sr = make_pipeline(StandardScaler(), lgbm)

estimators_list = [
                   ("gbm_sr", gbm_sr), 
                   ("xgb_sr", xgb_sr), 
                   ("lgbm_sr", lgbm_sr)
                  ]

X_train, X_test, y_train, y_test = split_data(train_features_df)
stack = StackingRegressor(estimators = estimators_list, n_jobs = -1, verbose = 3, passthrough = True)
stack.fit(X_train, np.log(y_train))

stack_preds = np.exp(stack.predict(X_test))
stack_rmsle = np.sqrt(mean_squared_log_error(y_true = y_test, y_pred = stack_preds))

In [49]:
stack_rmsle

0.13756978244959728

## Stack 2

**Base Models** used - 
- GBM with Robust Scaling
- XGBoost
- LightGBM with Standard Scaling

<br>

**Final Model** used - 
- RidgeCV(Default)

<br>

**Passthrough** = False

In [50]:
gbm_sr = make_pipeline(RobustScaler(), gbm)
xgb_sr = xgb
lgbm_sr = make_pipeline(StandardScaler(), lgbm)

estimators_list = [
                   ("gbm_sr", gbm_sr), 
                   ("xgb_sr", xgb_sr), 
                   ("lgbm_sr", lgbm_sr)
                  ]

X_train, X_test, y_train, y_test = split_data(train_features_df)
stack = StackingRegressor(estimators = estimators_list, n_jobs = -1, verbose = 1, passthrough = False)
stack.fit(X_train, np.log(y_train))

stack_preds = np.exp(stack.predict(X_test))
stack_rmsle = np.sqrt(mean_squared_log_error(y_true = y_test, y_pred = stack_preds))

In [51]:
stack_rmsle

0.13714132038424012

## Stack 3

**Base Models** used - 
- GBM with Robust Scaling
- XGBoost
- LightGBM with Standard Scaling

<br>

**Final Model** used -
- Lasso with StandardScaler

<br>

**Passthrough** = False


In [52]:
gbm_sr = make_pipeline(RobustScaler(), gbm)
xgb_sr = xgb
lgbm_sr = make_pipeline(StandardScaler(), lgbm)

estimators_list = [
                   ("gbm_sr", gbm_sr), 
                   ("xgb_sr", xgb_sr), 
                   ("lgbm_sr", lgbm_sr)
                  ]
lasso_sr = make_pipeline(StandardScaler(), Lasso(alpha = 0.0014, random_state = 42))

X_train, X_test, y_train, y_test = split_data(train_features_df)
stack = StackingRegressor(estimators = estimators_list, 
                          final_estimator = lasso_sr, 
                          n_jobs = -1, 
                          verbose = 1, 
                          passthrough = False)
stack.fit(X_train, np.log(y_train))

stack_preds = np.exp(stack.predict(X_test))
stack_rmsle = np.sqrt(mean_squared_log_error(y_true = y_test, y_pred = stack_preds))

In [53]:
stack_rmsle

0.13773670362742535

## Stack 4

**Base Models** used - 
- GBM
- XGBoost
- LightGBM

<br>

**Final Model** used -
- RidgeCV(Default)

<br>

**Passthrough** = True


In [56]:
gbm_sr = gbm
xgb_sr = xgb
lgbm_sr = lgbm

estimators_list = [
                   ("gbm_sr", gbm_sr), 
                   ("xgb_sr", xgb_sr), 
                   ("lgbm_sr", lgbm_sr)
                  ]
#lasso_sr = make_pipeline(StandardScaler(), Lasso(alpha = 0.0014, random_state = 42))

X_train, X_test, y_train, y_test = split_data(train_features_df)
stack = StackingRegressor(estimators = estimators_list,  
                          n_jobs = -1, 
                          verbose = 1, 
                          passthrough = True)
stack.fit(X_train, np.log(y_train))

stack_preds = np.exp(stack.predict(X_test))
stack_rmsle = np.sqrt(mean_squared_log_error(y_true = y_test, y_pred = stack_preds))

In [57]:
stack_rmsle

0.13755279460580574

# Submission

## Stack 1 - Passthrough = True

In [59]:
gbm_sr = gbm
xgb_sr = xgb
lgbm_sr = lgbm

estimators_list = [
                   ("gbm_sr", gbm_sr), 
                   ("xgb_sr", xgb_sr), 
                   ("lgbm_sr", lgbm_sr)
                  ]

X, y = train_features_df.drop(["SalePrice"], axis = 1), train_features_df["SalePrice"]
stack1 = StackingRegressor(estimators = estimators_list,  
                           n_jobs = -1, 
                           verbose = 1, 
                           passthrough = True)
stack1.fit(X.values, np.log(y.values))

stack1_preds = np.exp(stack1.predict(test_features_df.values))

In [63]:
sub5_stack_pass = pd.read_csv(url + "sample_submission.csv")
sub5_stack_pass["SalePrice"] = stack1_preds
sub5_stack_pass.to_csv("sub5_stack_pass.csv", index = False)

## Stack 2 - Passthrough = False

In [64]:
gbm_sr = gbm
xgb_sr = xgb
lgbm_sr = lgbm

estimators_list = [
                   ("gbm_sr", gbm_sr), 
                   ("xgb_sr", xgb_sr), 
                   ("lgbm_sr", lgbm_sr)
                  ]

X, y = train_features_df.drop(["SalePrice"], axis = 1), train_features_df["SalePrice"]
stack2 = StackingRegressor(estimators = estimators_list,  
                           n_jobs = -1, 
                           verbose = 1, 
                           passthrough = False)
stack2.fit(X.values, np.log(y.values))

stack2_preds = np.exp(stack2.predict(test_features_df.values))

In [68]:
sub5_stack = pd.read_csv(url + "sample_submission.csv")
sub5_stack["SalePrice"] = stack2_preds
sub5_stack.to_csv("sub5_stack.csv", index = False)